In [14]:
from pianoq.misc.mplc_writeup_imports import *
from pianoq.misc.mplc_lab_imports import * 

dir_path = r'G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror'
if not os.path.exists(dir_path):
    os.mkdir(dir_path)
    
dz = 5 
deadzone = 150e-6
plane_10_tilt = None 

# WFM

In [11]:
all_Us_path = r"G:\My Drive\Projects\MPLC\results\lab\2024_09_10_fixing_phases_different_Us\for_ronen_Haar_800.mat"
all_Us = loadmat(all_Us_path)['U1_all']
U_no = 1

timestamp = datetime.datetime.now().strftime('%Y_%m_%d_%H_%M_%S')
masks_path = fr'{dir_path}\{timestamp}_QKD_dead_zone_{deadzone*1e6}um_dz10_{dz}mm.masks'
mplc_res_path = fr'{dir_path}\{timestamp}_QKD_dead_zone_{deadzone*1e6}um_dz10_{dz}mm.mplcsim'
# mplc_sim = create_WFM_unitary_masks(all_Us[:, :, U_no], N_iterations=15, out_path=masks_path, 
#                                     dead_middle_zone=deadzone, last_plane_extra_dist=dz*1e-3)
mplc_sim = create_WFM_QKD_masks(N_iterations=15, out_path=masks_path, 
                                dead_middle_zone=deadzone, last_plane_extra_dist=dz*1e-3)
mplc_sim.res._calc_fidelity()
print(f'{mplc_sim.res.fidelity=}')

100%|██████████| 15/15 [03:34<00:00, 14.28s/it]


In [18]:
mplc_sim.res.show_field_intensity((np.abs(mplc_sim.res.input_modes)**2).sum(axis=0))
mplc_sim.res.show_field_intensity((np.abs(mplc_sim.res.output_modes)**2).sum(axis=0))

In [13]:
mplc_sim.res.show_all(2)

# Hardware

In [15]:
mplc = MPLCDevice(linear_tilts=True, plane_10_tilts=plane_10_tilt)
integration_time = 1
coin_window = 2e-9
tt = get_timetagger(coin_window=coin_window, integration_time=integration_time)

In [3]:
backlash = 0
wait_after_move = 0.2
mxs, mys = get_signal_motors(backlash=backlash, wait_after_move=wait_after_move)

backlash = 0
wait_after_move = 0.2
mxi, myi = get_idler_motors(backlash=backlash, wait_after_move=wait_after_move)

Got Zaber motors
Got Thorlabs motors


# Single counts

In [16]:
# Set MPLC 
# dummy old phases path meanwhile  
old_phases_path = rf'G:\My Drive\Projects\MPLC\results\lab\2024_10_30_make_mirror_work\2024_10_31_10_15_02_fine_new_mirror_from_zero_more.phases'
phases_path = rf"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_12_49_24.phases"
masks_path = fr"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_11_00_32_QKD_dead_zone_150.0um_dz10_5mm.masks"

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=old_phases_path)
mplc.plane_10_tilts = plane_10_tilt
mplc.load_masks(masks)
time.sleep(0.5)
mplc.restore_location()

In [ ]:
resolution = 1
integration_time = 1 

# Scan! 
scan_saveto_path = rf'{dir_path}\{tnow()}_signal_dead_zone_{deadzone}um_dz10_{dz}mm_pl10tilt_{plane_10_tilt}_res_{resolution}_d_mirror.scan'
start_x = 9.9
end_x = 10.6
start_y = 8.5
end_y = 10.5

x_pixels = 28 // resolution
y_pixels = 80 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path) 
tt = get_timetagger(integration_time=integration_time)  # important to have a new tt 
t1 = run_in_thread(scanner.scan, x_motor=mxs, y_motor=mys, ph=tt)

time.sleep(2)

scan_saveto_path = rf'{dir_path}\{tnow()}_idler_dead_zone_{deadzone}um_dz10_{dz}mm_pl10tilt_{plane_10_tilt}_res_{resolution}_d_mirror.scan'

start_x = 9.2
end_x = 9.8
start_y = 3.0
end_y = 5.0

x_pixels = 24 // resolution
y_pixels = 80 // resolution
pixel_size_x = 0.025 * resolution
pixel_size_y = 0.025 * resolution

scanner = PhotonScanner(integration_time, start_x, start_y, x_pixels, y_pixels, pixel_size_x, pixel_size_y, is_timetagger=True, coin_window=coin_window, saveto_path=scan_saveto_path)
tt2 = get_timetagger(integration_time=integration_time)
t2 = run_in_thread(scanner.scan, x_motor=mxi, y_motor=myi, ph=tt2)
t1.join()
send_telegram_message(f'Done one singles scan')
t2.join()    
send_telegram_message(f'Done second singles scan')

## Manual motors

In [46]:
mxi.move_absolute(9.542)
myi.move_absolute(4.652)
display(mxi.get_position())
display(myi.get_position())

9.54202

4.652

In [61]:
mxs.move_absolute(10.154)
mys.move_absolute(8.790)
display(mxs.get_position())
display(mys.get_position())

10.153983375

8.790003375000001

## Mark spots 

In [20]:
locs_signal_path = fr'{dir_path}\{tnow()}_s2_sig.locs'
locs_idler_path = fr'{dir_path}\{tnow()}_s1_idl.locs'

res_idl = ScanResult(fr'{dir_path}\2025_01_30_11_28_37_idler_dead_zone_0.00015um_dz10_5mm_pl10tilt_None_res_1_d_mirror.scan')
res_sig = ScanResult(fr'{dir_path}\2025_01_30_11_28_35_signal_dead_zone_0.00015um_dz10_5mm_pl10tilt_None_res_1_d_mirror.scan')
# res_idl.show_singles()
# res_sig.show_singles()
res_idl.get_xys(1, saveto_path=locs_idler_path)
res_sig.get_xys(2, saveto_path=locs_signal_path)

[(10.1340367965368, 8.798106060606061),
 (10.182738095238097, 9.144426406926408),
 (10.247673160173163, 9.512391774891777),
 (10.28555194805195, 9.820833333333335),
 (10.318019480519482, 10.150919913419916)]

# Phases

In [36]:
old_phases_path = rf"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_12_49_24.phases"
masks_path = fr"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_11_00_32_QKD_dead_zone_150.0um_dz10_5mm.masks"

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=None) # Need to be none so don't get confused! 

locs_idl = np.load(fr'{dir_path}\2025_01_30_12_35_24_s1_idl.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2025_01_30_12_35_24_s2_sig.locs')['locs']
mplc.load_masks(masks)
    
saveto_path = fr'{dir_path}\{tnow()}.phases'
integration_time = 30

old_phases_result = PhaseFinderResult(old_phases_path)
old_phases_result.reload()
initial_phases = old_phases_result.phases
print(f'{initial_phases=}')

pf = PhaseFinder(mplc=mplc, integration_time=integration_time, remote_tagger=True, saveto_path=saveto_path,
                 modes_to_keep=modes_to_keep, intial_phases=initial_phases, coin_window=tt.coin_window, no_hw_mode=True)

pf.time_tagger = tt
pf.time_tagger.set_integration_time(integration_time)


# i,j of corr matrix that is supposed to be strong 
i = 0
j = 0
mxi.move_absolute(locs_idl[i, 0])
myi.move_absolute(locs_idl[i, 1])
mxs.move_absolute(locs_sig[j, 0])
mys.move_absolute(locs_sig[j, 1])
mplc.restore_location()
time.sleep(1)

In [62]:
pf.find_phases(iterations=1)

# Correlations

In [63]:
phases_path = rf"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_13_20_58.phases"
masks_path = fr"G:\My Drive\Projects\MPLC\results\lab\2025_01_30_D_mirror\2025_01_30_11_00_32_QKD_dead_zone_150.0um_dz10_5mm.masks"

masks = get_good_masks(masks_path=masks_path, modes_to_keep=modes_to_keep, phases_path=phases_path)
mplc.plane_10_tilts = plane_10_tilt
mplc.load_masks(masks)
time.sleep(0.5)
mplc.restore_location()

In [66]:
backlash = 0.2
wait_after_move = 0.5
integration_time = 40
coin_window = 1e-9
tt = get_timetagger(integration_time=integration_time, coin_window=coin_window)

locs_idl = np.load(fr'{dir_path}\2025_01_30_12_35_24_s1_idl.locs')['locs']
locs_sig = np.load(fr'{dir_path}\2025_01_30_12_35_24_s2_sig.locs')['locs']

scan_path = fr'{dir_path}\{tnow()}_exp_{integration_time}_good_phases.dscan'

dps = DiscretePhotonScanner(locs_sig, locs_idl, integration_time=integration_time, remote_tagger=True, saveto_path=scan_path, backlash=backlash, wait_after_move=wait_after_move, coin_window=coin_window, no_hw_mode=True)

dps.m_sig_x, dps.m_sig_y, dps.m_idl_x, dps.m_idl_y, dps.time_tagger = mxs, mys, mxi, myi, tt
dps.time_tagger.set_integration_time(integration_time)
time.sleep(1)
dps.scan()